### Runs an extended Kalman filter using IHME SEIIR predictions along with measurement data of confirmed Covid-19 case counts (from New York Times data) and Facebook symptom data (loss of smell/taste, from Covid-19 Symptom Challenge) to generate updated 7-day predictions of case counts for counties in New York State.

Developed by the University of Washington team of Les Atlas, Abraham Flaxman and Michael Rhoads.

S - Susceptible
E - Exposed
I1 - Presymptomatic
I2 - Symptomatic
R - Recovered

In [3]:
#%load_ext autoreload
#%autoreload

In [4]:
#%reset

In [5]:
import math
import datetime

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

import data_sets
import seiir_compartmental

In [6]:
# functions to support the Kalman filtering
def get_predicts_prior(day, seiir):
    x_hat = np.array([[seiir['S'].loc[day]],
                      [seiir['E'].loc[day]],
                      [seiir['I1'].loc[day]],
                      [seiir['I2'].loc[day]],
                      [seiir['R'].loc[day]]])

    beta_k = seiir['beta'].loc[day]

    return x_hat, beta_k


def step_seiir(x_hat, constants, beta_k, days=7):
    s_dict = {'S': x_hat[0, 0],
              'E': x_hat[1, 0],
              'I1': x_hat[2, 0],
              'I2': x_hat[3, 0],
              'R': x_hat[4, 0]}

    s = pd.Series(s_dict)

    for i in range(days):
        infectious = s.loc['I1'] + s.loc['I2']
        s = seiir_compartmental.compartmental_covid_step(s, s.sum(),
                                                         infectious,
                                                         constants['alpha'],
                                                         beta_k,
                                                         constants['gamma1'],
                                                         constants['gamma2'],
                                                         constants['sigma'],
                                                         constants['theta'])
    x_hat_future_prior = np.array([[s.loc['S']],
                                   [s.loc['E']],
                                   [s.loc['I1']],
                                   [s.loc['I2']],
                                   [s.loc['R']]])

    return x_hat_future_prior


def predict_step(x_hat_k1_prior, P, Q, beta_k, constants):
    S = x_hat_k1_prior[0, 0]
    E = x_hat_k1_prior[1, 0]
    I1 = x_hat_k1_prior[2, 0]
    I2 = x_hat_k1_prior[3, 0]
    R = x_hat_k1_prior[4, 0]
    N = S + E + I1 + I2 + R
    alpha = constants['alpha']
    sigma = constants['sigma']
    gamma1 = constants['gamma1']
    gamma2 = constants['gamma2']

    part_f_S = np.array([[-beta_k * math.pow(I1 + I2, alpha) / N],
                         [beta_k * math.pow(I1 + I2, alpha) / N],
                         [0],
                         [0],
                         [0]])

    part_f_E = np.array([[0],
                         [-sigma],
                         [sigma],
                         [0],
                         [0]])

    part_f_I1 = np.array([[-alpha * beta_k * S * math.pow(I1+I2, alpha-1) / N],
                          [alpha * beta_k * S * math.pow(I1+I2, alpha-1) / N],
                          [-gamma1],
                          [gamma1],
                          [0]])

    part_f_I2 = np.array([[-alpha * beta_k * S * math.pow(I1+I2, alpha-1) / N],
                          [alpha * beta_k * S * math.pow(I1+I2, alpha-1) / N],
                          [0],
                          [-gamma2],
                          [gamma2]])

    part_f_R = np.array([[0],
                         [0],
                         [0],
                         [0],
                         [0]])

    # 5x5
    f_jacob = np.concatenate([part_f_S, part_f_E, part_f_I1, part_f_I2,
                              part_f_R], axis=1)

    # 5x5
    # P_k1_prior = f_jacob * P * f_jacob^T + Q
    P_k1_prior = np.matmul(np.matmul(f_jacob, P), np.transpose(f_jacob)) + Q
    return P_k1_prior


def update_step(x_hat, x_hat_k1, P_k1, Rn, rho1, rho2, z_k, measure):
    # 5x5
    ep = 10**-8
    if measure is None:
        H = np.array([[ep, 0, 0, 0, 0],
                      [0, ep, 0, 0, 0],
                      [0, 0, ep, 0, 0],
                      [0, 0, 0, rho2, 0],
                      [0, 0, 0, 0, ep]])
    else:
                
        H = np.array([[ep, 0, 0, 0, 0],
                      [0, ep, 0, 0, 0],
                      [0, 0, ep, rho1, 0],
                      [0, 0, 0, rho2, 0],
                      [0, 0, 0, 0, ep]])
    
    
    # Si = H * P_k1 * H^T + Rn
    Si = np.matmul(np.matmul(H, P_k1), np.transpose(H)) + Rn

    # K_new = P_k1 * H^T * Si^(-1)
    K_new = np.matmul(np.matmul(P_k1, np.transpose(H)), np.linalg.inv(Si))
    y_new = np.matmul(H, x_hat)

    # 5x1
    diff = z_k - y_new

    x_hat_k1_post = x_hat_k1 + np.matmul(K_new, diff)

    P_k1_post = P_k1 - np.matmul(np.matmul(K_new, Si), np.transpose(K_new))


    return x_hat_k1_post, P_k1_post, diff, K_new[3, 3]


def create_data_sets():
    seiir_fl = pd.read_csv(r'data/seiir_projections/florida_proj.csv', header=0,
                            index_col='date', parse_dates=True)

    seiir_ny = pd.read_csv(r'data/seiir_projections/new_york_proj.csv', header=0,
                            index_col='date', parse_dates=True)
    
    smell_data = data_sets.create_symptom_df()
    
    case_data = data_sets.create_case_df_county()
    
    return seiir_fl, seiir_ny, smell_data, case_data


def get_smell_data(fips, fb_data):

    if fips == 'New York City':
        nyc_fips = ['36005', '36061', '36047', '36085']
        fb_data_geo = fb_data.loc[(slice(None), '36081'), :].copy()
        fb_data_geo = fb_data_geo.mean(level='date')

        for borough in nyc_fips:
            fb_data_geo += fb_data.loc[(slice(None), borough), :].copy().mean(level='date')

    else:
        fb_data_geo = fb_data.loc[(slice(None), fips), :].copy()

        # collapse down to a single index column (date)
        fb_data_geo.index = fb_data_geo.index.droplevel([0, 1])

    return fb_data_geo
    


def calc_fb_ma7(fb_data):
    """
    Returns a Pandas series
    """
    # the fb_data is a DataFrame while the case_data is a Series
    fb_ma7 = fb_data.rolling(window=7).mean()
    fb_ma7 = fb_ma7.iloc[6:, :]
    prop_ma7 = fb_ma7['num_stl'].div(fb_ma7['n'])

    return prop_ma7, fb_ma7['n'].copy(), fb_ma7['num_stl'].copy()


def calc_mse(prediction, actual):
    """
    Inputs should be two Pandas Series of same length.
    Outputs a float.
    """
    err = prediction - actual
    sum_sq_err = (err**2).sum()
    mse = sum_sq_err / err.count()
    return mse


def create_hh_data():
    # bring in new data
    full_data = pd.read_csv(r'data/from_challenge/overall-county.csv', header=0, dtype={'fips': 'str', 'pct_hh_cli': 'float64'},
                            parse_dates=[0])
    time_start = full_data['date'].min()
    time_end = full_data['date'].max()
    full_data.set_index(['fips', 'date'], inplace=True)
    full_data.sort_index(inplace=True)

    # derive count of survey respondents with household members having covid symptoms
    full_data['num_hh_cli'] = full_data['n'].mul(full_data['pct_hh_cli']/100.).round()
    full_data['num_hh_cli'] = full_data['num_hh_cli'].astype('int64')

    # group by county and date
    data_of_interest = full_data[['n', 'num_hh_cli']].copy().groupby(level=(0, 1)).sum()
    idx = pd.IndexSlice

    # create full date range
    date_rng = pd.date_range(time_start, time_end)
    iterables = [data_of_interest.index.levels[0], date_rng]
    new_index = pd.MultiIndex.from_product(iterables, names=['fips', 'date'])

    data_of_interest = data_of_interest.reindex(index=new_index)
    # this will have NaN values in the new index entries for which there was no
    # previous data -- fill them upon extracting a particular county
    
    return data_of_interest


In [7]:
def create_ny_county_list():
    case_data = data_sets.create_case_df_county()
    ny_counties = case_data[case_data['state'] == 'New York']['county'].unique()
    return ny_counties

In [8]:
seiir_fl, seiir_ny, smell_data, case_data = create_data_sets()

hh_data = create_hh_data()

In [23]:
def run_county(the_state, county_name, seiir, smell_data, hh_data, case_data, measure='smell', K0=datetime.date(2020, 4, 18), delay=6):
    """
    Arguments:
        measure -- potential values are: 'smell', 'hh', or None
    """
    # set constants
    #K0 = datetime.date(2020, 4, 12)

    if county_name == 'New York City':
        the_county = county_name
    else:
        the_county = data_sets.get_fips(the_state, county_name)

    constants = {
        'alpha': 0.948786,
        'gamma1': 0.500000,
        'gamma2': 0.662215,
        'sigma': 0.266635,
        'theta': 6.000000
        }

    # set initial values for Kalman filter parameters
    P_mult = 1
    Q_mult = 1

    # Rn is the R noise covariance matrix; it remains constant thru the stepping of the
    # Kalman filter
    Rn_mult = 5*10**-8

    Rn_22 = 10000
    Rn_32 = 1000

    Rn_23 = 1000
    Rn_33 = 100

    Rn = Rn_mult * np.array([[0, 0, 0, 0, 0],
                             [0, 0, 0, 0, 0],
                             [0, 0, Rn_22, Rn_23, 0],
                             [0, 0, Rn_32, Rn_33, 0],
                             [0, 0, 0, 0, 0]])

    Q = Q_mult * np.eye(5)
    P = P_mult * np.eye(5)
    
    if the_county == 'New York City':
        county_pop = 0
        for each in ['36081', '36005', '36061', '36047', '36085']:
            this_count, state_pop = data_sets.get_pops(each)
            county_pop += this_count
    else:
        county_pop, state_pop = data_sets.get_pops(the_county)

    b = county_pop / state_pop

    # generate data
    case_data_geo = case_data.loc[the_county]['case_rate'].copy()
    smell_data_geo = get_smell_data(the_county, smell_data)
    

    if measure == 'hh':
        idx = pd.IndexSlice
        
        if the_county == 'New York City':
            nyc_fips = ['36005', '36061', '36047', '36085']
            hh_cli = hh_data.loc[idx['36081', :], :].loc['36081'].copy()
            hh_cli.fillna(method='pad', inplace=True)

            for borough in nyc_fips:
                bor_hh_cli = hh_data.loc[idx[borough, :], :].loc[borough].copy()
                bor_hh_cli.fillna(method='pad', inplace=True)
                hh_cli += bor_hh_cli

        else:
            hh_cli = hh_data.loc[idx[the_county, :], :].loc[the_county].copy()
            hh_cli.fillna(method='pad', inplace=True)

        # calculate moving averages on the fb and case data
        hh_cli_ma7 = hh_cli.rolling(window=7).mean()
        hh_cli_ma7 = hh_cli_ma7.iloc[6:, :]
        num_survey_ma7 = hh_cli_ma7['num_hh_cli']
        prop_cli_ma7 = num_survey_ma7.div(hh_cli_ma7['n'])
        
    elif measure == 'smell':
        prop_ma7, n_ma7, num_survey_ma7 = calc_fb_ma7(smell_data_geo)
        
    else:
        num_survey_ma7 = None


    case_ma7 = case_data_geo.rolling(window=7).mean()
    case_ma7_all = case_ma7.iloc[6:]
    

    # get starting compartment values for the state level
    x_hat_state_k0, beta_k0 = get_predicts_prior(K0, seiir)

    # convert to the county level
    x_hat_k0 = b * x_hat_state_k0

    I2_county = x_hat_k0[3, 0]


    rho1 = 0.0001
    rho2 = case_ma7_all.loc['2020-04-12'] / I2_county


    # create empty dictionaries to hold the estimated values
    case_est = {}
    seiir_pred = {}

    diff_rat = {}

    K_val_dict = {}


    # Original data run ----------------
    start = K0
    d = start

    while d <= datetime.date(2020, 10, 23):    

    # each cycle of the while loop executes a step

        # get state level compartments
        x_hat_state_k, beta_k = get_predicts_prior(d, seiir)

        # step the state level compartments 7 days forward
        x_hat_state_k1 = step_seiir(x_hat_state_k, constants, beta_k)

        # convert the state level compartments to county level values
        x_hat_k = b * x_hat_state_k
        x_hat_k1 = b * x_hat_state_k1

        indexDate = d + datetime.timedelta(days=7)
        # store seiir prediction before it's modified by Kalman filter
        seiir_pred[indexDate] = x_hat_k1[3, 0]

        # get measurements for current day
        if measure == 'smell':
            z_k = np.array([[0],
                            [0],
                            [prop_ma7.loc[d - datetime.timedelta(days=delay)]],
                            [case_ma7_all.loc[d]],
                            [0]])    

        elif measure == 'hh':
            z_k = np.array([[0],
                            [0],
                            [prop_cli_ma7.loc[d - datetime.timedelta(days=delay)]],
                            [case_ma7_all.loc[d]],
                            [0]])
            
        else:
            z_k = np.array([[0],
                            [0],
                            [0],
                            [case_ma7_all.loc[d]],
                            [0]])

        # predict step using the stepped fwd SEIIR compartment values 
        P = predict_step(x_hat_k1, P, Q, beta_k, constants)

        # update step
        x_hat_post, P_post, the_diff, K_val = update_step(x_hat_k, x_hat_k1, P, Rn,
                                         rho1, rho2, z_k, measure)


        # store estimated values for proportion and case rate
        K_val_dict[indexDate] = K_val
        case_est[indexDate] = rho2 * x_hat_post[3, 0]


        diff_rat[indexDate] = the_diff[2, 0] / the_diff[3, 0]

        # update the P and d
        P = P_post
        d += datetime.timedelta(days=1)

    # create pandas series of the estimated case rate
    predicted_case = pd.Series(case_est)
    predicted_seiir_prior = pd.Series(seiir_pred)
    K_val_series = pd.Series(K_val_dict)
    
    return predicted_case, predicted_seiir_prior, case_ma7_all, num_survey_ma7, K_val_series

In [ ]:
#predicted_case, predicted_seiir_prior, case_ma7_all, num_survey_ma7, K_vals = run_county('NY', 'Westchester', seiir_ny, smell_data, hh_data, case_data, measure='hh', K0=datetime.date(2020, 4, 18), delay=6)

In [24]:
K0 = datetime.date(2020, 4, 18)

ny_counties = create_ny_county_list()
df_index = []
err_data = []

predictions_case_only = pd.DataFrame()
predictions_smell_0delay = pd.DataFrame()
predictions_smell_6delay = pd.DataFrame()
predictions_hh_0delay = pd.DataFrame()
predictions_hh_6delay = pd.DataFrame()

left = K0 + datetime.timedelta(days=7)
right = datetime.date(2020, 10, 23)

for each in ny_counties:
    if each == 'New York City':
        fips = 'New York City'
    else:
        fips = data_sets.get_fips('NY', each)
        if (fips not in hh_data.index.levels[0]) or (fips not in smell_data.index.levels[1]):
            continue
    
    print('Starting', fips)
    
    county_data = {}
    
    predicted_case, predicted_seiir_prior, case_ma7_all, num_survey_ma7, K_vals = run_county('NY', each, seiir_ny, smell_data, hh_data, case_data, measure=None, K0=K0, delay=0)
    county_data['mse_confirmed-only'] = calc_mse(predicted_case.loc[left:right], case_ma7_all.loc[left:right])
    predictions_case_only[each] = predicted_case
    

    predicted_case, predicted_seiir_prior, case_ma7_all, num_survey_ma7, K_vals = run_county('NY', each, seiir_ny, smell_data, hh_data, case_data, measure='smell', K0=K0, delay=0)
    county_data['mse_smell-0delay'] = calc_mse(predicted_case.loc[left:right], case_ma7_all.loc[left:right])
    predictions_smell_0delay[each] = predicted_case
    

    predicted_case, predicted_seiir_prior, case_ma7_all, num_survey_ma7, K_vals = run_county('NY', each, seiir_ny, smell_data, hh_data, case_data, measure='smell', K0=K0, delay=6)
    county_data['mse_smell-6delay'] = calc_mse(predicted_case.loc[left:right], case_ma7_all.loc[left:right])
    predictions_smell_6delay[each] = predicted_case
    

    predicted_case, predicted_seiir_prior, case_ma7_all, num_survey_ma7, K_vals = run_county('NY', each, seiir_ny, smell_data, hh_data, case_data, measure='hh', K0=K0, delay=0)
    county_data['mse_hh-0delay'] = calc_mse(predicted_case.loc[left:right], case_ma7_all.loc[left:right])
    predictions_hh_0delay[each] = predicted_case
    

    predicted_case, predicted_seiir_prior, case_ma7_all, num_survey_ma7, K_vals = run_county('NY', each, seiir_ny, smell_data, hh_data, case_data, measure='hh', K0=K0, delay=6)
    county_data['mse_hh-6delay'] = calc_mse(predicted_case.loc[left:right], case_ma7_all.loc[left:right])
    predictions_hh_6delay[each] = predicted_case
    
    df_index.append(fips)
    err_data.append(county_data)
    print('Complete with', fips)

Starting New York City
Complete with New York City


In [28]:
err_data

[{'mse_confirmed-only': 30503.884158190707,
  'mse_smell-0delay': 30918.89834954391,
  'mse_smell-6delay': 31120.081178154087,
  'mse_hh-0delay': 30582.06780755661,
  'mse_hh-6delay': 31113.70747269597}]

In [ ]:
['36005', '36061', '36047', '36085', '36081']


In [22]:
if '36005' not in smell_data.index.levels[1]:
    print('not in there')

In [ ]:
if ('36123' not in hh_data.index.levels[0]) or ('36123' not in smell_data.index.levels[1]):
    print('not in there!')

In [27]:
df_index

['New York City']

In [26]:
predictions_case_only


,New York City
2020-04-25,3294.172813
2020-04-26,3108.061050
2020-04-27,2972.785847
2020-04-28,2822.001753
2020-04-29,2205.671129
...,...
2020-10-26,645.237743
2020-10-27,619.298111
2020-10-28,674.681808
2020-10-29,678.659192


In [ ]:
for each in ny_counties:
    fips = data_sets.get_fips('NY', each)
    print(each, fips)

In [ ]:
predictions_case_only

In [ ]:
# error computations

In [ ]:
# compute squared error for the overlapping time period -------------------------
d = datetime.date(2020, 10, 23)
start = datetime.date(2020, 4, 18)

left = predicted_seiir_prior.index[0]
right = d

seiir_err = case_ma7_all.loc[left:right] - predicted_seiir_prior[left:right]
seiir_sum_sq_err = (seiir_err**2).sum()
seiir_mse = seiir_sum_sq_err / seiir_err.count()

kalman_err = predicted_case.loc[left:right] - case_ma7_all.loc[left:right]
kalman_sum_sq_err = (kalman_err**2).sum()
kalman_mse = kalman_sum_sq_err / kalman_err.count()


print('SSE between seiir forecast and case rate:', seiir_sum_sq_err)
print('MSE between seiir forecast and measured case rate:', seiir_mse)
print()
print('SSE between kalman forecast and case rate:', kalman_sum_sq_err)
print('MSE between kalman forecast and measured case rate:', kalman_mse)

In [ ]:
predicted_case

In [ ]:
calc_mse(predicted_case.loc[left:right], case_ma7_all.loc[left:right])

In [ ]:
test_df = pd.DataFrame()

In [ ]:
test_df['test'] = predicted_case

In [ ]:
test_df

In [ ]:
output_df = pd.DataFrame()
output_df[the_county] = predicted_case

In [ ]:
output_df.to_csv(r'output/predicted_case_example.csv')

# All plotting code below this

In [ ]:
xlim_left = None
xlim_right = None

leftylim_low = None
leftylim_high = None

rightylim_low = None
rightylim_high = None

xtick_size = 14
xlabel_size = 14

In [ ]:
# plot findings -- multiple plots

# Plotting constants and variables ----------------

plt.style.use('seaborn-whitegrid')
matplotlib.rcParams.update({'font.size': 18})
purple = '#33016F'
gold = '#9E7A27'
gray = '#797979'
width = 4
%matplotlib qt

tick_end = predicted_seiir_prior.index[-1]

week_interval = pd.date_range(start=start, end=tick_end, freq='W')
week_interval = [x.to_pydatetime().date() for x in week_interval]



In [ ]:
# multiple plots -----------
fig1, ax11 = plt.subplots(1)
plt.sca(ax11)
plt.plot(case_ma7_all.loc[start:d].index, case_ma7_all.loc[start:d], label='Confirmed Case Count', c=gold,
         linewidth=width)
plt.xticks(week_interval, rotation=30, ha='right', rotation_mode='anchor', fontsize=xtick_size)
plt.ylabel('Number of Cases per Day')
plt.legend(loc='upper left')
plt.ylim(leftylim_low, leftylim_high)
plt.xlim(xlim_left, xlim_right)


fig2, ax21 = plt.subplots(1)
plt.sca(ax21)
plt.plot(case_ma7_all.loc[start:d].index, case_ma7_all.loc[start:d], label='Confirmed Case Count', c=gold,
         linewidth=width)
plt.plot(predicted_seiir_prior.index, predicted_seiir_prior,
         label='IHME 7-day Forecast', c=gray, linewidth=width)
plt.xticks(week_interval, rotation=30, ha='right', rotation_mode='anchor', fontsize=xtick_size)
plt.ylabel('Number of Cases per Day')
plt.legend(loc='upper left')
plt.ylim(leftylim_low, leftylim_high)
plt.xlim(xlim_left, xlim_right)




fig3, ax31 = plt.subplots(1)
plt.sca(ax31)
plt.plot(case_ma7_all.loc[start:d].index, case_ma7_all.loc[start:d], label='Confirmed Case Count', c=gold,
         linewidth=width)
plt.plot(predicted_seiir_prior.index, predicted_seiir_prior,
         label='IHME 7-day Forecast', c=gray, linewidth=width)

ax32 = ax31.twinx()
plt.sca(ax32)
plt.plot(num_stl_ma7.loc[start:d].index, num_stl_ma7.loc[start:d], c='red', label='FB Positive Symptoms, Smell/Taste Loss',
         linewidth=width)
plt.grid(axis='y', linestyle=':')

plt.ylabel('Number of Positive Symptom Response per Day')
plt.legend(loc='upper right')
plt.ylim(rightylim_low, rightylim_high)


plt.sca(ax31)
plt.xticks(week_interval, rotation=30, ha='right', rotation_mode='anchor', fontsize=xtick_size)
plt.ylabel('Number of Cases per Day')

plt.legend(loc='upper left')
plt.ylim(leftylim_low, leftylim_high)
plt.xlim(xlim_left, xlim_right)






In [ ]:
fig4, ax41 = plt.subplots(1)
plt.sca(ax41)
plt.plot(case_ma7_all.loc[start:d].index, case_ma7_all.loc[start:d], label='Confirmed Case Count', c=gold,
         linewidth=width)
plt.plot(predicted_seiir_prior.index, predicted_seiir_prior,
         label='IHME 7-day Forecast', c=gray, linewidth=width)
#naieve estimate
naive_start = start + datetime.timedelta(days=7)
naive_d = d + datetime.timedelta(days=7)
plt.plot(case_ma7_all.loc[naive_start:naive_d].index, case_ma7_all.loc[start:d], label='Naive guess', c='orange')

ax42 = ax41.twinx()
plt.sca(ax42)
plt.plot(num_survey_ma7.loc[start:d].index, num_survey_ma7.loc[start:d], c='red', label='FB Positive Symptoms, Smell/Taste Loss',
         linewidth=width)
plt.grid(axis='y', linestyle=':')

plt.ylabel('Number of Positive Household Symptom Response per Day')
plt.legend(loc='upper right')
plt.ylim(rightylim_low, rightylim_high)


plt.sca(ax41)
plt.plot(predicted_case.loc[start:].index, predicted_case.loc[start:], label='Our 7-Day Forecast',
         c=purple, linewidth=width)

plt.xticks(week_interval, rotation=30, ha='right', rotation_mode='anchor', fontsize=xtick_size)
plt.ylabel('Number of Cases per Day')

plt.legend(loc='upper left')
plt.ylim(leftylim_low, leftylim_high)
plt.xlim(xlim_left, xlim_right)

In [ ]:
case_ma7_all.loc[d]


In [ ]:
plt.grid()

In [ ]:
## special end date

# plot findings -- multiple plots

# Plotting constants and variables ----------------
matplotlib.rcParams.update({'font.size': 18})
plt.style.use('seaborn-whitegrid')
purple = '#33016F'
gold = '#9E7A27'
gray = '#797979'
width = 4
%matplotlib qt

tick_end = predicted_seiir_prior.index[-1]

week_interval = pd.date_range(start=start, end=tick_end, freq='W')
week_interval = [x.to_pydatetime().date() for x in week_interval]


fig1, ax11 = plt.subplots(1)
plt.sca(ax11)
plt.plot(case_ma7_all.loc[start:tick_end].index, case_ma7_all.loc[start:tick_end], label='Confirmed Case Count', c=gold,
         linewidth=width)
plt.xticks(week_interval, rotation=30, ha='right', rotation_mode='anchor', fontsize=xtick_size)
plt.ylabel('Number of Cases per Day')
plt.legend(loc='upper left')
plt.ylim(leftylim_low, leftylim_high)
plt.xlim(xlim_left, xlim_right)


fig2, ax21 = plt.subplots(1)
plt.sca(ax21)
plt.plot(case_ma7_all.loc[start:tick_end].index, case_ma7_all.loc[start:tick_end], label='Confirmed Case Count', c=gold,
         linewidth=width)
plt.plot(predicted_seiir_prior.index, predicted_seiir_prior,
         label='IHME 7-day Forecast', c=gray, linewidth=width)
plt.xticks(week_interval, rotation=30, ha='right', rotation_mode='anchor', fontsize=xtick_size)
plt.ylabel('Number of Cases per Day')
plt.legend(loc='upper left')
plt.ylim(leftylim_low, leftylim_high)
plt.xlim(xlim_left, xlim_right)




fig3, ax31 = plt.subplots(1)
plt.sca(ax31)
plt.plot(case_ma7_all.loc[start:tick_end].index, case_ma7_all.loc[start:tick_end], label='Confirmed Case Count', c=gold,
         linewidth=width)
plt.plot(predicted_seiir_prior.index, predicted_seiir_prior,
         label='IHME 7-day Forecast', c=gray, linewidth=width)

ax32 = ax31.twinx()
plt.sca(ax32)
plt.plot(num_stl_ma7.loc[start:tick_end].index, num_stl_ma7.loc[start:tick_end], c='red', label='FB Positive Symptoms, Smell/Taste Loss',
         linewidth=width)
plt.grid(axis='y', linestyle=':')

plt.ylabel('Number of Positive Symptom Response per Day')
plt.legend(loc='upper right')
plt.ylim(rightylim_low, rightylim_high)


plt.sca(ax31)
plt.xticks(week_interval, rotation=30, ha='right', rotation_mode='anchor', fontsize=xtick_size)
plt.ylabel('Number of Cases per Day')

plt.legend(loc='upper left')
plt.ylim(leftylim_low, leftylim_high)
plt.xlim(xlim_left, xlim_right)



fig4, ax41 = plt.subplots(1)
plt.sca(ax41)
plt.plot(case_ma7_all.loc[start:tick_end].index, case_ma7_all.loc[start:tick_end], label='Confirmed Case Count', c=gold,
         linewidth=width)
plt.plot(predicted_seiir_prior.index, predicted_seiir_prior,
         label='IHME 7-day Forecast', c=gray, linewidth=width)

ax42 = ax41.twinx()
plt.sca(ax42)
plt.plot(num_stl_ma7.loc[start:tick_end].index, num_stl_ma7.loc[start:tick_end], c='red', label='FB Positive Symptoms, Smell/Taste Loss',
         linewidth=width)
plt.grid(axis='y', linestyle=':')

plt.ylabel('Number of Positive Symptom Response per Day')
plt.legend(loc='upper right')
plt.ylim(rightylim_low, rightylim_high)


plt.sca(ax41)
plt.plot(predicted_case.loc[start:].index, predicted_case.loc[start:], label='Our 7-Day Forecast',
         c=purple, linewidth=width)

plt.xticks(week_interval, rotation=30, ha='right', rotation_mode='anchor', fontsize=xtick_size)
plt.ylabel('Number of Cases per Day')

plt.legend(loc='upper left')
plt.ylim(leftylim_low, leftylim_high)
plt.xlim(xlim_left, xlim_right)


In [ ]:
# NYC plot settings

xlim_left = datetime.date(2020, 4, 2)
xlim_right = datetime.date(2020, 11, 3)

leftylim_low = -700
leftylim_high = 12000

rightylim_low = -5
rightylim_high = 85

xtick_size = 14
xlabel_size = 14

In [ ]:
# Nassau plot settings
xlim_left = datetime.date(2020, 4, 2)
xlim_right = datetime.date(2020, 11, 3)

leftylim_low = -40
leftylim_high = 1650

rightylim_low = -.3
rightylim_high = 12

xtick_size = 14
xlabel_size = 14

In [ ]:
# Westchester plot settings
xlim_left = datetime.date(2020, 4, 2)
xlim_right = datetime.date(2020, 11, 3)

leftylim_low = -28
leftylim_high = 1000

rightylim_low = -.2
rightylim_high = 7

xtick_size = 14
xlabel_size = 14

In [ ]:
# Albany plot settings
xlim_left = datetime.date(2020, 4, 2)
xlim_right = datetime.date(2020, 11, 3)

leftylim_low = -3
leftylim_high = 75

rightylim_low = -.125
rightylim_high = 3.1

xtick_size = 14
xlabel_size = 14

In [ ]:
# Erie plot settings
xlim_left = datetime.date(2020, 4, 2)
xlim_right = datetime.date(2020, 10, 13)

leftylim_low = -10
leftylim_high = 250

rightylim_low = -.5
rightylim_high = 12.5

xtick_size = 14
xlabel_size = 14